In [113]:
import base64
from collections import Counter
import secrets
import numpy as np
from Crypto.Cipher import AES

In [114]:
datas = np.loadtxt("https://cryptopals.com/static/challenge-data/20.txt", dtype="str")

In [115]:
def generate_random_bytes(bytes_length):
    random_bytes = secrets.token_bytes(bytes_length)
    return random_bytes

BLOCK_LENGTH = 16
KEY = generate_random_bytes(BLOCK_LENGTH)

In [116]:
def crypt_CTR(message, key=KEY, block_len=BLOCK_LENGTH):
    message_len = len(message)
    crypted_message = b""
    for i in range((message_len - 1) // block_len + 1):
        num = i
        nonce = b"\x00" * 8
        block_index = num.to_bytes(8, byteorder="little")
        cipher = AES.new(key, AES.MODE_ECB)
        if i < (message_len - 1) // block_len:
            encrypted_block = message[i*block_len:(i+1)*block_len]
        else:
            encrypted_block = message[i*block_len:]
        key_stream = cipher.encrypt(nonce+block_index)
        decrypted_block = bytes([c1^c2 for c1, c2 in zip(encrypted_block, key_stream[:len(encrypted_block)])])
        crypted_message += decrypted_block
    return crypted_message

In [117]:
datas = [base64.b64decode(data) for data in datas]
encrypted_datas = [crypt_CTR(message) for message in datas]

In [ ]:
def find_common_size(datas):
    common_size = min(len(data) for data in datas)
    return common_size

In [119]:
common_size = find_common_size(encrypted_datas)

clipped_datas = [encrypted_data[:common_size] for encrypted_data in encrypted_datas]
concat_clipped_datas = b"".join(clipped_datas)

In [ ]:
def assign_score(output_string):
    string_score = 0
    freq = [' ', 'e', 't', 'a']
    for letter in output_string:
        if letter in freq:
            string_score += 1
    return string_score

def find_single_key(message):
    string_score = 0
    key = None
    for k in range(256):
        res = []
        for i in range(0, len(message)):
            res.append(chr(k ^ message[i]))
        if assign_score("".join(res)) > string_score:
            key = k
            string_score = assign_score("".join(res))
    return key

def find_repeated_key(message, keysize):
    assert len(message) % keysize == 0
    keys = []
    for i in range(keysize):
        keys.append(find_single_key(message[i::keysize]))
    return keys

def decrypt_repeated_key_xor(message, keysize):
    res = []
    keys = find_repeated_key(message, keysize)
    for i in range(len(message) // keysize):
        for j in range(keysize):
            res.append((message[i*keysize+j])^keys[j])

    return bytes(res), keys

In [122]:
decrypted_clipped, keys = decrypt_repeated_key_xor(concat_clipped_datas, common_size)